In [7]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
today_date = '2026-02-13'

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 9, Finished, Available, Finished, False)

In [8]:
from pyspark.sql.functions import col 
df = spark.read.format('delta').load("abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/tblsales_silver").filter(col('processed_date')==str(today_date))


StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 10, Finished, Available, Finished, False)

In [9]:
display(df)

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 11, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 38c8725e-41b6-4bc9-a721-f7fc736288a3)

In [6]:
df.printSchema()

StatementMeta(, bbd350b9-491c-456d-a303-5b4e13541923, 8, Finished, Available, Finished, False)

root
 |-- Row_ID: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)
 |-- Ship_Mode: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Postal_Code: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping_Cost: double (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Month: string (nu

### Creating Dimensional table

## Dim_Product_table

In [1]:
from pyspark.sql.types import *

from delta.tables import DeltaTable


v_structure_Schema = StructType([
    StructField('Customer_ID',StringType(),True) ,
    StructField('Customer_Name',StringType(),True) ,
    StructField('Segment',StringType(),True) ,
    StructField('Postal_Code',StringType(),True) ,
    StructField('City',StringType(),True) ,
    StructField('State',StringType(),True) ,
    StructField('Country',StringType(),True) ,
    StructField('Region',StringType(),True) ,
    StructField('Market',StringType(),True) 
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Customer').addColumns(v_structure_Schema).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 3, Finished, Available, Finished, False)

In [2]:
from pyspark.sql.types import *


v_structure_Schema1 = StructType([
    StructField('Product_ID',StringType(),True) ,
    StructField('Catagory',StringType(),True) ,
    StructField('Sub_Catagory',StringType(),True) ,
    StructField('Product_Name',StringType(),True) 
])

DeltaTable.createIfNotExists(spark).tableName('Dim_Product').addColumns(v_structure_Schema1).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 4, Finished, Available, Finished, False)

In [33]:
v_dim_fact_Schema = StructType([
    StructField("Row_ID",StringType(),True),
    StructField("Order_ID",StringType(),True),
    StructField("Customer_ID",StringType(),True),
    StructField("Customer_Name",StringType(),True),
    StructField("Order_Date",DateType(),True),
    StructField("Ship_Date",DateType(),True),
    StructField("Ship_Mode",StringType(),True),
    StructField("Sales",DoubleType(),True),     
    StructField("Quantity",IntegerType(),True), 
    StructField("Discount",DoubleType(),True), 
    StructField("Profit",DoubleType(),True),  
    StructField("Shipping_Cost",DoubleType(),True), 
    StructField("Order_Priority",StringType(),True), 
    StructField("Month",StringType(),True), 
    StructField("Year",StringType(),True),
    StructField("Processed_Date",StringType(),True),
    StructField("Product_ID",StringType(),True),
    StructField("Delivery_Date",StringType(),True),    
    StructField("Profit_Margin",StringType(),True)  
])

DeltaTable.createIfNotExists(spark).tableName('Fact_Sales').addColumns(v_dim_fact_Schema).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 35, Finished, Available, Finished, False)

In [10]:
df_Dim_customer = (df.select(
    'Customer_ID',
    'Customer_Name' ,
    'Segment',
    'Postal_Code',
    'City',
    'State' ,
    'Country' ,
    'Region' ,
    'Market').dropDuplicates())

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 12, Finished, Available, Finished, False)

In [11]:
from delta.tables import *
fabric_dim_customer = "abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/dim_customer"
dim_customer = DeltaTable.forPath(spark,fabric_dim_customer)


dim_customer.alias('target').merge(
df_Dim_customer.alias('source'),'target.Customer_ID = source.Customer_ID'
).whenMatchedUpdate(
    set = {'target.Customer_Name':'source.Customer_Name',
    'target.Segment':'source.Segment',
    'target.Postal_Code':'source.Postal_Code',
    'target.City':'source.City',
    'target.State':'source.State',
    'target.Country':'source.Country',
    'target.Region':'source.Region',
    'target.Market':'source.Market'
}).whenNotMatchedInsert(
     values={
        'Customer_ID':'source.Customer_ID',
        'Customer_Name':'source.Customer_Name',
    'Segment':'source.Segment',
    'Postal_Code':'source.Postal_Code',
    'City':'source.City',
    'State':'source.State',
    'Country':'source.Country',
    'Region':'source.Region',
    'Market':'source.Market'
     }
).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 13, Finished, Available, Finished, False)

In [12]:
%%sql
select * from dim_customer

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 14, Finished, Available, Finished, False)

<Spark SQL result set with 849 rows and 9 fields>

In [18]:
history_df = dim_customer.history()
display(history_df)
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
display(operation_metrics)
rows_inserted = operation_metrics.get("numTargetRowsInserted",0)
rows_updated = operation_metrics.get("numTargetRowsUpdated",0)
rows_deleted = operation_metrics.get("numTargetRowsDeleted",0)
rows_impacted = int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of Table',dim_customer.toDF().count())
print(f"Rows Inserted : {rows_inserted} ")
print(f"Rows Updated : {rows_updated} ")
print(f"Rows deleted : {rows_deleted} ")
print(f"Rows impacted : {rows_impacted} ")



StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 20, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 2be63479-3695-4d69-b54f-02caa040ed50)

{'numOutputRows': '849',
 'numTargetBytesAdded': '36499',
 'numTargetRowsInserted': '849',
 'numTargetFilesAdded': '1',
 'numTargetRowsMatchedDeleted': '0',
 'numTargetFilesRemoved': '0',
 'numTargetRowsMatchedUpdated': '0',
 'executionTimeMs': '5376',
 'numTargetDeletionVectorsUpdated': '0',
 'numTargetRowsCopied': '0',
 'rewriteTimeMs': '1716',
 'numTargetRowsUpdated': '0',
 'numTargetDeletionVectorsRemoved': '0',
 'numTargetRowsDeleted': '0',
 'scanTimeMs': '3059',
 'numSourceRows': '849',
 'numTargetDeletionVectorsAdded': '0',
 'numTargetChangeFilesAdded': '0',
 'numTargetRowsNotMatchedBySourceUpdated': '0',
 'numTargetRowsNotMatchedBySourceDeleted': '0',
 'numTargetBytesRemoved': '0'}

Total rows of Table 849
Rows Inserted : 849 
Rows Updated : 0 
Rows deleted : 0 
Rows impacted : 849 


# **Loading to Product Table**

In [22]:
df_Dim_product = (df.select(
    'Product_ID',
    'Category' ,
    'Sub_Category',
    'Product_Name').dropDuplicates())

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 24, Finished, Available, Finished, False)

In [25]:
from delta.tables import *
fabric_dim_Product = "abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/dim_product"
dim_Product = DeltaTable.forPath(spark,fabric_dim_Product)


dim_Product.alias('target').merge(
df_Dim_product.alias('source'),'target.Product_ID = source.Product_ID'
).whenMatchedUpdate(
    set = {
    'target.Catagory':'source.Category',
    'target.Sub_Catagory':'source.Sub_Category',
    'target.Product_Name':'source.Product_Name'
}).whenNotMatchedInsert(
     values={
'Product_ID':'source.Product_ID',
    'Catagory':'source.Category',
    'Sub_Catagory':'source.Sub_Category',
    'Product_Name':'source.Product_Name'
     }
).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 27, Finished, Available, Finished, False)

In [26]:
history_df = dim_Product.history()
display(history_df)
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
display(operation_metrics)
rows_inserted = operation_metrics.get("numTargetRowsInserted",0)
rows_updated = operation_metrics.get("numTargetRowsUpdated",0)
rows_deleted = operation_metrics.get("numTargetRowsDeleted",0)
rows_impacted = int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of Table',dim_customer.toDF().count())
print(f"Rows Inserted : {rows_inserted} ")
print(f"Rows Updated : {rows_updated} ")
print(f"Rows deleted : {rows_deleted} ")
print(f"Rows impacted : {rows_impacted} ")

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 28, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, f69e7773-0973-4fdc-82ed-0c19057edf74)

{'numOutputRows': '1185',
 'numTargetBytesAdded': '32532',
 'numTargetRowsInserted': '1185',
 'numTargetFilesAdded': '1',
 'numTargetRowsMatchedDeleted': '0',
 'numTargetFilesRemoved': '0',
 'numTargetRowsMatchedUpdated': '0',
 'executionTimeMs': '3371',
 'numTargetDeletionVectorsUpdated': '0',
 'numTargetRowsCopied': '0',
 'rewriteTimeMs': '346',
 'numTargetRowsUpdated': '0',
 'numTargetDeletionVectorsRemoved': '0',
 'numTargetRowsDeleted': '0',
 'scanTimeMs': '1882',
 'numSourceRows': '1185',
 'numTargetDeletionVectorsAdded': '0',
 'numTargetChangeFilesAdded': '0',
 'numTargetRowsNotMatchedBySourceUpdated': '0',
 'numTargetRowsNotMatchedBySourceDeleted': '0',
 'numTargetBytesRemoved': '0'}

Total rows of Table 849
Rows Inserted : 1185 
Rows Updated : 0 
Rows deleted : 0 
Rows impacted : 1185 


# **Loading to Fact Table**

In [34]:
df_Fact_sales = (df.select(
    'Row_ID',
    'Order_ID',
    'Customer_ID',
    'Customer_Name',
    'Order_Date',
    'Ship_Date',
    'Ship_Mode',
    'Sales',     
    'Quantity', 
    'Discount', 
    'Profit',  
    'Shipping_Cost', 
    'Order_Priority', 
    'Month', 
    'Year',
    'Processed_Date',
    'Product_ID',
    'Delivery_Days',    
    'Profit_Margin').dropDuplicates())

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 36, Finished, Available, Finished, False)

In [35]:
from delta.tables import *
fabric_dim_Fact_Sales = "abfss://Sales_workspace@onelake.dfs.fabric.microsoft.com/Sales_lakehouse.Lakehouse/Tables/dbo/fact_sales"
dim_fact = DeltaTable.forPath(spark,fabric_dim_Fact_Sales)


dim_fact.alias('target').merge(
df_Fact_sales.alias('source'),'target.Order_ID = source.Order_ID'
).whenMatchedUpdate(
    set = {'target.Row_ID':'source.Row_ID',
    #'target.Order_ID':'source.Order_ID',
    'target.Customer_ID':'source.Customer_ID',
    'target.Customer_Name':'source.Customer_Name',
    'target.Order_Date':'source.Order_Date',
    'target.Ship_Date':'source.Ship_Date',
    'target.Ship_Mode':'source.Ship_Mode',
    'target.Sales':'source.Sales',
    'target.Quantity':'source.Quantity',
    'target.Discount':'source.Discount',
    'target.Profit':'source.Profit',
    'target.Shipping_Cost':'source.Shipping_Cost',
    'target.Order_Priority':'source.Order_Priority',
    'target.Month':'source.Month',
    'target.Year':'source.Year',
    'target.Processed_Date':'source.Processed_Date',
    'target.Delivery_Date':'source.Delivery_Days',
    'target.Profit_Margin':'source.Profit_Margin'
}).whenNotMatchedInsert(
     values={
'target.Row_ID':'source.Row_ID',
    'target.Order_ID':'source.Order_ID',
    'target.Customer_ID':'source.Customer_ID',
    'target.Customer_Name':'source.Customer_Name',
    'target.Order_Date':'source.Order_Date',
    'target.Ship_Date':'source.Ship_Date',
    'target.Ship_Mode':'source.Ship_Mode',
    'target.Sales':'source.Sales',

    'target.Quantity':'source.Quantity',
    'target.Discount':'source.Discount',
    'target.Profit':'source.Profit',
    'target.Shipping_Cost':'source.Shipping_Cost',
    'target.Order_Priority':'source.Order_Priority',
    'target.Month':'source.Month',
    'target.Year':'source.Year',
    'target.Processed_Date':'source.Processed_Date',
    'target.Delivery_Date':'source.Delivery_Days',
    'target.Profit_Margin':'source.Profit_Margin'
     }
).execute()

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 37, Finished, Available, Finished, False)

In [36]:
history_df = dim_fact.history()
display(history_df)
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
display(operation_metrics)
rows_inserted = operation_metrics.get("numTargetRowsInserted",0)
rows_updated = operation_metrics.get("numTargetRowsUpdated",0)
rows_deleted = operation_metrics.get("numTargetRowsDeleted",0)
rows_impacted = int(rows_inserted)+int(rows_updated)+int(rows_deleted)

print('Total rows of Table',dim_fact.toDF().count())
print(f"Rows Inserted : {rows_inserted} ")
print(f"Rows Updated : {rows_updated} ")
print(f"Rows deleted : {rows_deleted} ")
print(f"Rows impacted : {rows_impacted} ")

StatementMeta(, 749eeb32-5aae-4a71-8991-b0c91525296f, 38, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 47970fc3-2385-45ba-9db8-c26d58f1b750)

{'numOutputRows': '1675',
 'numTargetBytesAdded': '111211',
 'numTargetRowsInserted': '1675',
 'numTargetFilesAdded': '1',
 'numTargetRowsMatchedDeleted': '0',
 'numTargetFilesRemoved': '0',
 'numTargetRowsMatchedUpdated': '0',
 'executionTimeMs': '2711',
 'numTargetDeletionVectorsUpdated': '0',
 'numTargetRowsCopied': '0',
 'rewriteTimeMs': '413',
 'numTargetRowsUpdated': '0',
 'numTargetDeletionVectorsRemoved': '0',
 'numTargetRowsDeleted': '0',
 'scanTimeMs': '1789',
 'numSourceRows': '1675',
 'numTargetDeletionVectorsAdded': '0',
 'numTargetChangeFilesAdded': '0',
 'numTargetRowsNotMatchedBySourceUpdated': '0',
 'numTargetRowsNotMatchedBySourceDeleted': '0',
 'numTargetBytesRemoved': '0'}

Total rows of Table 1675
Rows Inserted : 1675 
Rows Updated : 0 
Rows deleted : 0 
Rows impacted : 1675 


In [1]:
%%sql
select * from dim_product

StatementMeta(, c401dd81-3e29-4e86-a867-c49154c56fed, 2, Finished, Available, Finished, False)

<Spark SQL result set with 1000 rows and 4 fields>